<a href="https://colab.research.google.com/github/omar4646/My_repository/blob/main/princ_Econ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd
import numpy as np
!pip install pyreadr
import statsmodels.formula.api as smf


In [88]:
import pyreadr

chemical = pyreadr.read_r('/content/chemical2.rda')

In [89]:
chemical = chemical['chemical2']

In [90]:
chemical

,year,firm,lsales,lcapital,llabor,sk_labor,lmaterials,foreign,export,intangibles,ownership
0,2004,1,10.421390,9.848661,5.308268,0.267327,9.964441,0.25,1,1,0
1,2005,1,10.879330,10.065050,5.438079,0.267327,10.453630,0.25,1,1,0
2,2006,1,11.282740,10.093900,5.493062,0.267327,10.836850,0.25,1,1,0
3,2004,2,9.355739,8.774158,4.025352,0.000000,8.940368,0.00,0,0,0
4,2005,2,9.313799,8.874867,4.025352,0.000000,9.021598,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
595,2005,199,9.758462,8.872066,4.700480,0.016667,9.233275,0.00,0,0,0
596,2006,199,9.895959,9.052516,4.787492,0.016667,9.248406,0.00,0,0,0
597,2004,200,8.836374,7.649693,4.007333,0.000000,8.326758,0.00,0,1,0
598,2005,200,9.126959,8.143517,4.007333,0.000000,8.830981,0.00,0,0,0


In [91]:
chemical["d_lnSALES"] = chemical.groupby('firm')["lsales"].diff(periods=1)
chemical["d_lnCAPITAL"] = chemical.groupby('firm')["lcapital"].diff(periods=1)
chemical["d_lnLABOR"] = chemical.groupby('firm')["llabor"].diff(periods=1)

formula = 'd_lnSALES ~ d_lnCAPITAL + d_lnLABOR'
chemical_diff = chemical.dropna(subset=["d_lnSALES", "d_lnCAPITAL", "d_lnLABOR"]).copy()
res_fd = smf.ols(formula, data=chemical_diff).fit(
    cov_type="cluster", cov_kwds={"groups": chemical_diff['firm']}
)
print("\n------ First-difference OLS results (clustered SE by id) ------")
print(res_fd.summary())



------ First-difference OLS results (clustered SE by id) ------
                            OLS Regression Results                            
Dep. Variable:              d_lnSALES   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     16.06
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           3.42e-07
Time:                        12:16:10   Log-Likelihood:                -160.65
No. Observations:                 400   AIC:                             327.3
Df Residuals:                     397   BIC:                             339.3
Df Model:                           2                                         
Covariance Type:              cluster                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------

In [92]:
vars_to_demean = ["lsales", "lcapital", "llabor"]
for v in vars_to_demean:
    mean_name = f"{v}_mean"
    difference = f"differ_{v}"
    chemical[mean_name] = chemical.groupby('firm')[v].transform("mean")
    chemical[difference] = chemical[v] - chemical[mean_name]
# ---- Within (demeaned) OLS: tilde_y ~ tilde_cap + tilde_lab  (no intercept) ----
formula = "differ_lsales ~ differ_lcapital + differ_llabor - 1"
print("\nRunning within OLS (formula):", formula)
res_within = smf.ols(formula, data=chemical).fit(cov_type="cluster", cov_kwds={"groups": chemical['firm']})
print(res_within.summary())


Running within OLS (formula): differ_lsales ~ differ_lcapital + differ_llabor - 1
                                 OLS Regression Results                                
Dep. Variable:          differ_lsales   R-squared (uncentered):                   0.125
Model:                            OLS   Adj. R-squared (uncentered):              0.122
Method:                 Least Squares   F-statistic:                              19.49
Date:                Thu, 11 Sep 2025   Prob (F-statistic):                    1.86e-08
Time:                        12:16:10   Log-Likelihood:                          5.8503
No. Observations:                 600   AIC:                                     -7.701
Df Residuals:                     598   BIC:                                      1.093
Df Model:                           2                                                  
Covariance Type:              cluster                                                  
                      coef    std err

Pooled OLS Model (Restricted Model)

The pooled OLS model assumes no fixed effects (i.e., ignores the unobserved heterogeneity).

In [93]:
import statsmodels.formula.api as smf
df = chemical.set_index(['firm', 'year'])
df = df.sort_index()
# Estimate the Pooled OLS model (no fixed effects)
pooled_model = smf.ols('lsales ~ lcapital + llabor', data=df).fit()

# Pooled OLS results
print(pooled_model.summary())


                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                     298.0
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.79e-90
Time:                        12:16:10   Log-Likelihood:                -748.36
No. Observations:                 600   AIC:                             1503.
Df Residuals:                     597   BIC:                             1516.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.8797      0.171     34.368      0.0

Fixed Effects Model (Unrestricted Model)

Now, estimate the fixed effects model, which accounts for unobserved heterogeneity using within transformation.

In [94]:
# !pip install linearmodels
from linearmodels.panel import PanelOLS

# Fixed Effects model
fe_model = PanelOLS.from_formula('lsales ~ lcapital + llabor + EntityEffects', data=df)
fe_res = fe_model.fit()

# Fixed Effects results
print(fe_res)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 lsales   R-squared:                        0.1252
Estimator:                   PanelOLS   R-squared (Between):              0.4106
No. Observations:                 600   R-squared (Within):               0.1252
Date:                Thu, Sep 11 2025   R-squared (Overall):              0.4104
Time:                        12:16:10   Log-likelihood                    5.8503
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      28.491
Entities:                         200   P-value                           0.0000
Avg Obs:                       3.0000   Distribution:                   F(2,398)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             28.491
                            

Compute SSE for Both Models

Next, you will need to compute the sum of squared residuals (SSE) for both the fixed effects and pooled OLS models.

In [95]:
# Sum of squared residuals for Pooled OLS model
SSE_R = sum((pooled_model.fittedvalues - df['lsales'])**2)

print(f"SSE_R (Pooled OLS): {SSE_R}")


SSE_R (Pooled OLS): 425.63655733503504


In [96]:
# Sum of squared residuals for Pooled OLS model
SSE_R = sum((pooled_model.fittedvalues - df['lsales'])**2)

# Sum of squared residuals for Fixed Effects model (using the correct method to access fitted values)
# Align indices before computing the difference
fe_predicted = fe_res.predict()
SSE_U = sum((fe_predicted['fitted_values'] - df['lsales'])**2)


print(f"SSE_R (Pooled OLS): {SSE_R}")
print(f"SSE_U (Fixed Effects): {SSE_U}")

SSE_R (Pooled OLS): 425.63655733503504
SSE_U (Fixed Effects): 34956.62973313458


In [97]:
df

lsales   lcapital    llabor  sk_labor  lmaterials  foreign  \
firm year                                                                  
1    2004  10.421390   9.848661  5.308268  0.267327    9.964441     0.25   
     2005  10.879330  10.065050  5.438079  0.267327   10.453630     0.25   
     2006  11.282740  10.093900  5.493062  0.267327   10.836850     0.25   
2    2004   9.355739   8.774158  4.025352  0.000000    8.940368     0.00   
     2005   9.313799   8.874867  4.025352  0.000000    9.021598     0.00   
...              ...        ...       ...       ...         ...      ...   
199  2005   9.758462   8.872066  4.700480  0.016667    9.233275     0.00   
     2006   9.895959   9.052516  4.787492  0.016667    9.248406     0.00   
200  2004   8.836374   7.649693  4.007333  0.000000    8.326758     0.00   
     2005   9.126959   8.143517  4.007333  0.000000    8.830981     0.00   
     2006   9.879707   8.470101  4.007333  0.000000    9.661861     0.00   

           export  intangibles  ownership  d_lnSALES  d_lnCAPITAL  d_lnLABOR  \
firm year                                                                      
1    2004       1            1          0        NaN          NaN        NaN   
     2005       1            1          0   0.457940     0.216389   0.129811   
     2006       1            1          0   0.403410     0.028850   0.054983   
2    2004       0            0          0        NaN          NaN        NaN   
     2005       0            0          0  -0.041940     0.100709   0.000000   
...           ...          ...        ...        ...          ...        ...   
199  2005       0            0          0  -0.581668     0.816908  -0.087012   
     2006       0            0          0   0.137497     0.180450   0.087012   
200  2004       0            1          0        NaN          NaN        NaN   
     2005       0            0          0   0.290585     0.493824   0.000000   
     2006       0            1          0   0.752748     0.326584   0.000000   

           lsales_mean  differ_lsales  lcapital_mean  differ_lcapital  \
firm year                                                               
1    2004    10.861153      -0.439763      10.002537        -0.153876   
     2005    10.861153       0.018177      10.002537         0.062513   
     2006    10.861153       0.421587      10.002537         0.091363   
2    2004     9.414841      -0.059102       8.841297        -0.067139   
     2005     9.414841      -0.101042       8.841297         0.033570   
...                ...            ...            ...              ...   
199  2005     9.998184      -0.239722       8.659913         0.212153   
     2006     9.998184      -0.102225       8.659913         0.392603   
200  2004     9.281013      -0.444639       8.087770        -0.438077   
     2005     9.281013      -0.154054       8.087770         0.055747   
     2006     9.281013       0.598694       8.087770         0.382331   

           llabor_mean  differ_llabor  
firm year                              
1    2004     5.413136      -0.104868  
     2005     5.413136       0.024943  
     2006     5.413136       0.079926  
2    2004     4.031252      -0.005900  
     2005     4.031252      -0.005900  
...                ...            ...  
199  2005     4.758488      -0.058008  
     2006     4.758488       0.029004  
200  2004     4.007333       0.000000  
     2005     4.007333       0.000000  
     2006     4.007333       0.000000  

[600 rows x 18 columns]

In [99]:
chemical_three = pyreadr.read_r('/content/chemical3.rda')
chemical3 = chemical_three['chemical3']

In [100]:
chemical3

,year,firm,lsales,lcapital,llabor,sk_labor,lmaterials,foreign,export,intangibles,ownership
0,2004,1,10.421390,9.848661,5.308268,0.267327,9.964441,0.25,1,1,0
1,2005,1,10.879330,10.065050,5.438079,0.267327,10.453630,0.25,1,1,0
2,2006,1,11.282740,10.093900,5.493062,0.267327,10.836850,0.25,1,1,0
3,2004,2,9.355739,8.774158,4.025352,0.000000,8.940368,0.00,0,0,0
4,2005,2,9.313799,8.874867,4.025352,0.000000,9.021598,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2995,2005,999,11.646140,8.720950,4.997212,0.387755,11.167050,0.00,0,0,0
2996,2006,999,11.849400,9.740969,5.010635,0.387755,11.430090,0.00,0,0,0
2997,2004,1000,11.898800,11.339080,5.971262,0.145408,11.663720,0.00,1,1,0
2998,2005,1000,12.062720,11.497810,5.945421,0.145408,11.823320,0.00,1,1,0


In [101]:
# 2) Specify formula (pooled OLS) ----------------------------------------
formula = 'lsales ~ lcapital + llabor'   # intercept included by default

# 3) Pooled OLS - conventional (non-robust) -------------------------------
pooled = smf.ols(formula, data=chemical3).fit()   # conventional OLS
print("\nConventional OLS summary:")
print(pooled.summary())

# 4) Pooled OLS - heteroskedasticity-robust (HC1) -------------------------
pooled_hc = pooled.get_robustcov_results(cov_type='HC1')   # or 'HC3'
print("\nHeteroskedasticity-robust (HC1) summary:")
print(pooled_hc.summary())

# 5) Pooled OLS - cluster-robust by firm ---------------------------------
pooled_cl = pooled.get_robustcov_results(cov_type='cluster',
                                         groups=chemical3['firm'])
print("\nCluster-robust (cluster by firm) summary:")
print(pooled_cl.summary())


Conventional OLS summary:
                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.558
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     1892.
Date:                Thu, 11 Sep 2025   Prob (F-statistic):               0.00
Time:                        12:23:50   Log-Likelihood:                -3836.8
No. Observations:                3000   AIC:                             7680.
Df Residuals:                    2997   BIC:                             7698.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.5408    